# PDRPy - praca domowa 3

## Jakub Kobojek

## Ładowanie danych i pakietów
Najpierw należy zaimportować wszystkie niezbędne pakiety i otworzyć połączenie z bazą danych.
Pliki z danymi muszą byc w tym samym folderze co ten plik. Proszę upewnić się, że wszystkie pakiety są już zainstalowane.

In [6]:
import os
import pandas as pd
import numpy as np
import tempfile
import sqlite3

Votes  = pd.read_csv("Votes.csv.gz", compression='gzip')
Posts  = pd.read_csv("Posts.csv.gz", compression='gzip')
Users  = pd.read_csv("Users.csv.gz", compression='gzip')
Badges = pd.read_csv("Badges.csv.gz", compression='gzip')

database = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(database): 
    os.remove(database)

connection = sqlite3.connect(database) 
Posts.to_sql("Posts", connection)
Users.to_sql("Users", connection)
Votes.to_sql("Votes", connection)
Badges.to_sql("Badges", connection)

193592

Do porównywania rezultatów będę korzystać z funkcji `equals`.

## Zapytanie SQL 1

### Rozwiązanie referencyjne:

In [7]:
ref1 = pd.read_sql_query(
    """
    SELECT Name, COUNT(*) AS Number, MIN(Class) AS BestClass
    FROM Badges
    GROUP BY Name
    ORDER BY Number Desc
    LIMIT 10
    """, 
    connection)
ref1

,Name,Number,BestClass
0,Autobiographer,24564,3
1,Student,23930,3
2,Supporter,17826,3
3,Popular Question,14840,3
4,Nice Answer,12406,3
5,Yearling,11281,2
6,Teacher,9746,3
7,Editor,9115,3
8,Notable Question,8613,2
9,Scholar,7224,3


### Rozwiązanie z pandas

In [24]:
pd1 = Badges.groupby(['Name'], as_index=False).agg(Number=('Name','count'), BestClass=('Class', 'min')).sort_values(by='Number', ascending=False).head(10)
pd1

,Name,Number,BestClass
5,Autobiographer,24564,3
74,Student,23930,3
76,Supporter,17826,3
54,Popular Question,14840,3
48,Nice Answer,12406,3
84,Yearling,11281,2
81,Teacher,9746,3
23,Editor,9115,3
50,Notable Question,8613,2
67,Scholar,7224,3


## Test identyczności

In [9]:
pd1.equals(ref1)

False

## Zapytanie SQL 2
Zapytanie to zwraca 10 najczęściej występujących lokalizacji użytkowników, którzy utworzyli przynajmniej jeden post. Dodatkowo zwracana jest liczba ich wystapięń.


### Rozwiązanie referencyjne:

In [10]:
ref2 = pd.read_sql_query(
    """
    SELECT Location, COUNT(*) AS Count
    FROM (
        SELECT Posts.OwnerUserId, Users.Id, Users.Location
        FROM Users
        JOIN Posts ON Users.Id = Posts.OwnerUserId
        )
    WHERE Location NOT IN ('')
    GROUP BY Location
    ORDER BY Count DESC
    LIMIT 10
    """, 
    connection)
ref2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


### Rozwiązanie z pandas

In [11]:
merged = pd.merge(Users[["Id","Location"]], Posts["OwnerUserId"], left_on = 'Id', right_on = 'OwnerUserId')
grouped = merged.dropna(subset = ["Location"]).groupby(["Location"]).size().reset_index(name = 'Count')
pd2 = grouped.sort_values(by = ['Count'], ascending = False).reset_index(drop = True).head(10)
pd2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


### Test identyczności

In [12]:
pd2.equals(ref2)

True

### Komentarz
Zapytanie to było łatwe w implementacji, gdyż wymagało jedynie połączenia tabel i grupowania. Widzimy już tutaj intuicyjność pakietu `pandas`. Korzystanie z niego jest podobne do korzytania z SQL.

## Zapytanie SQL 3
Kolejne zapytanie wyświetla podstawowe informacje o pierwszych dziesięciu użytkownikach, którzy mają najwyższą średnią liczbę odpowiedzi pod swoimi postami

### Rozwiązanie referencyjne:

In [13]:
ref3 = pd.read_sql_query(
    """
    SELECT
        Users.AccountId,
        Users.DisplayName,
        Users.Location,
        AVG(PostAuth.AnswersCount) as AverageAnswersCount
    FROM
        (
        SELECT
            AnsCount.AnswersCount,
            Posts.Id,
            Posts.OwnerUserId
        FROM 
            (
                SELECT Posts.ParentId, COUNT(*) AS AnswersCount
                FROM Posts
                WHERE Posts.PostTypeId = 2
                GROUP BY Posts.ParentId
            ) AS AnsCount
            JOIN Posts ON Posts.Id = AnsCount.ParentId
        ) AS PostAuth
    JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
    GROUP BY OwnerUserId
    ORDER BY AverageAnswersCount DESC
    LIMIT 10
    """, 
    connection)
ref3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,None,8.0
7,42364.0,Petrogad,None,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


### Rozwiązanie z pandas

In [22]:
answers = Posts.where(Posts['PostTypeId'] == 2).groupby(['ParentId'], as_index=False).agg(AnswersCount=('ParentId', 'count'))
PostAuth = answers.merge(Posts, left_on='ParentId', right_on='Id')[['AnswersCount', 'Id', 'OwnerUserId']]
avg = PostAuth.merge(Users, left_on='OwnerUserId', right_on='AccountId').groupby(['OwnerUserId'], as_index=False).agg(AverageAnswersCount=("AnswersCount", "mean"))
pd3 = avg.merge(Users, left_on='OwnerUserId', right_on='AccountId')[['AccountId', 'DisplayName', 'Location', 'AverageAnswersCount']]\
            .sort_values(['AverageAnswersCount', 'AccountId'] , ascending=[False, False]).head(10)
pd3.index = range(0, 10)
pd3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,NaN,8.0
7,42364.0,Petrogad,NaN,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


### Test identyczności

In [15]:
pd3.equals(ref3)

True

### Komentarz
To zapytanie wymgało agregacji i łączenia tabel, co okazało sie dosyć trudne w `pandas`. Poradziłem sobie z tym licząc najpierw średnią liczbę odpowiedzi w każdej grupie a potem wykonując `join` z potrzebnymi kolumnami z uprzednio przefiltrowanej tabeli. 

## Zapytanie SQL 4
Pokazuje ono tytuł postu, który otrzymał najwięcej pozytywnych ocen w każdym roku. Ponadto zwraca liczbę tych ocen.

### Rozwiązanie referencyjne:

In [16]:
ref4 = pd.read_sql_query(
    """
    SELECT
        Posts.Title,
        UpVotesPerYear.Year,
        MAX(UpVotesPerYear.Count) AS Count
    FROM 
        (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
    JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
    WHERE Posts.PostTypeId=1
    GROUP BY Year
    ORDER BY Year ASC
    """, 
    connection)
ref4

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


### Rozwiązanie z pandas

In [23]:
Votes['Year'] = Votes['CreationDate'].str.slice(start = 0, stop = 4)
UpVotesPerYear = Votes[Votes['VoteTypeId'] == 2].groupby(['PostId','Year']).size().reset_index(name = 'Count')
merged = pd.merge(Posts, UpVotesPerYear, left_on  = 'Id', right_on = 'PostId')
titles = merged[merged['PostTypeId'] == 1][['Title','Year','Count']]
pd4 = titles.sort_values(['Year', 'Count'], ascending=[True, False]).groupby('Year').first().reset_index()
pd4 = pd4.reindex(columns = ['Title','Year','Count'])
pd4

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


### Test identyczności

In [18]:
pd4.equals(ref4)

True

In [19]:
connection.close()

### Komentarz
Zapytanie to bylo dla mnie łatwiejsze do implementacji przy użyciu `pandas` niz w R. Przede wszystkim dlatego, że można było wykorzystać przydatne operacje na stringach takie jak `str.slice`. Dzięki temu rozwiązałem to zapytanie w sposób znacznie szybszy i czystszy niz w R. Poza tym dużo łatwiej bylo znaleźć maksimum każdej grupy. Wystarczyło posortować i wziąc pierwszą wartość.

## Podsumowanie
Pomimo niewielkiego doświadczenia z pakietem `pandas` (miałem z nim styczność jedynie przy PD2), względnie łatwo poradziłem sobie z zadaniami. Na zapytanie nr 5 nie starczyło mi niestety czasu. Łatwosc mogła wynikać między innymi z podobności (składniowej, logicznej) odpowiedników w R i `pandas`. Jedyna różnica którą odnotowałem była konieczność resetowania indeksow po sortowaniu lub agregacji ramek danych. Poza tym bardzo pozytywnie oceniam doświadczenia związane z korzystaniem `Jupyter Notebook`.